# atomman.lammps.atom_dump.load(data, prop_info=None)

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-09-02

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The atom_dump.load function writes a system's information to a LAMMPS dump style file. The dump file format provides a concise way of storing a large number of atoms and atom properties. In converting between atomman and dump files, this function allows for full control over the property name conversions and unit control in the form of a JSON data model.

Equivalently, a dump file can be loaded by calling [atomman.load](atomman.load.ipynb) using the 'atom_dump' style. However, this function does not return a list of elements as that information is not included in dump files. (OK, it can be. Update later.) 

Argument:

- __data__ = file name, file-like object or string to read data from.

Keyword Argument:

- __prop_info__ = the data model defining how to relate the LAMMPS atomic attributes to the atomman atomic properties. If not specified, will try reading [fname].json.  If that doesn't work, a defaut model will be created based on the system and saved to [fname].json.

The underlying code can be found in [atomman/lammps/atom_dump.py](https://github.com/usnistgov/atomman/blob/master/atomman/lammps/atom_dump.py).

- - -

## Demonstration

Library imports

In [1]:
import atomman as am
import atomman.lammps as lmp
import numpy as np
import os

Create a demonstration dump file

In [2]:
with open('test.dump', 'w') as f:
    f.write("""ITEM: TIMESTEP
10000
ITEM: NUMBER OF ATOMS
32
ITEM: BOX BOUNDS pp pp pp
-0.0051871 7.04519
-0.000725292 7.04073
0.00106248 7.03894
ITEM: ATOMS id type x y z c_voro[1] c_voro[2] c_voro[3] c_stress[1] c_stress[2] c_stress[3] c_stress[4] c_stress[5] c_stress[6] 
1 1 0.34266 0.360855 0.36442 10.9751 15 26.4002 400382 -172369 361946 618490 404704 -320814 
2 1 2.1331 2.08739 0.364684 10.8585 14 26.2102 -615610 -230235 -309261 -51268.4 -22109.6 162304 
3 1 2.13011 0.356301 2.10647 10.7928 12 26.1077 -348083 -910871 -1.09017e+006 -378108 -14530.8 17524 
4 1 0.371264 2.14855 2.11487 10.8736 14 26.2371 -274887 -631508 -200777 91647.5 -110065 42194.8 
5 1 3.87914 0.336744 0.33682 10.8095 13 26.1342 -675768 -782205 -574469 280706 387178 -88306.6 
6 1 5.61718 2.12982 0.353137 10.9855 15 26.4155 634348 23563.8 352267 410632 -16999 89704.6 
7 1 5.64899 0.364151 2.10255 10.8934 13 26.2708 -377029 366661 -568931 -241658 -167956 -98843 
8 1 3.85763 2.09815 2.1219 10.8531 15 26.2035 -617312 -504639 -133137 82641.5 -305560 464891 
9 1 0.384493 3.86697 0.325359 10.9793 15 26.4068 762617 48876.4 -167497 -214166 604294 144525 
10 1 2.0794 5.67942 0.352479 10.9463 14 26.3537 -214649 -108288 53498.9 -272125 263911 -150107 
11 1 2.13615 3.8549 2.11903 10.9158 12 26.3081 -920203 369708 86175.3 181708 -20025.3 -179370 
12 1 0.368441 5.61418 2.11555 10.9311 14 26.3286 319290 169884 -445718 -97229.3 534560 -117761 
13 1 3.86316 3.8555 0.3695 10.9498 14 26.3592 -63914.2 48865.3 204479 -469176 772963 280291 
14 1 5.6071 5.61263 0.378892 10.9225 14 26.3164 -541094 619378 -532424 61303.2 500721 -352070 
15 1 5.65718 3.84845 2.1288 10.9447 14 26.3534 681429 -612603 28238.8 414261 -595882 -125175 
16 1 3.90279 5.65552 2.11917 10.8978 14 26.2785 -196415 82602.2 -837962 -52218 437772 -340591 
17 1 0.336733 0.346837 3.81881 10.9622 15 26.3778 -122284 7297.48 424914 6595.3 -438374 -217850 
18 1 2.09121 2.09103 3.88121 10.8248 14 26.1582 -607215 -321585 -826987 -215378 -24560.5 185736 
19 1 2.12442 0.338879 5.61191 10.8877 14 26.2626 570537 -890472 -514505 -129617 116092 136761 
20 1 0.362742 2.10803 5.63359 11.0239 14 26.4766 652102 609589 265711 -292428 303493 36423.4 
21 1 3.87914 0.335321 3.85591 10.8012 14 26.1222 -876425 -544486 -925326 -379683 -444495 -336783 
22 1 5.63331 2.10933 3.8781 10.9536 14 26.367 766226 -81820.7 -142462 -555.931 13348.6 486009 
23 1 5.59491 0.361244 5.61175 10.9649 13 26.3864 -277954 -14497.9 511873 227853 -10587.5 246844 
24 1 3.8594 2.10978 5.62638 10.8691 14 26.2291 -615773 -297935 -62400 25807.9 -10532.1 260447 
25 1 0.363466 3.87847 3.8738 10.9435 15 26.3476 125934 240235 -173885 366409 -786668 226420 
26 1 2.12211 5.66215 3.88061 10.9393 14 26.3457 282321 -674646 388013 -83610.5 -368395 -219128 
27 1 2.0934 3.87711 5.62107 10.9594 14 26.3778 -491800 1.03667e+006 -235943 -277648 716577 5646.56 
28 1 0.312444 5.66299 5.667 11.0392 15 26.4996 668414 561654 418506 45491.6 -411767 -131362 
29 1 3.8367 3.87183 3.87901 10.9319 15 26.3311 -147109 461054 -246050 -2179.35 -521350 55022.1 
30 1 5.63246 5.61437 3.85241 10.9248 13 26.3194 -57984.6 149025 -103228 219048 -358931 -128483 
31 1 5.60137 3.87556 5.6307 10.9684 14 26.3895 414195 -209028 328312 221033 -14988.5 90695.5 
32 1 3.89648 5.62805 5.65321 10.8717 14 26.234 -428645 -775321 19892.5 361894 -485759 -489471 
""")

### Reading dump file without prop_info

In [3]:
system = am.lammps.atom_dump.load('test.dump')

Show Box, atom types and atom positions 

In [4]:
print system

avect =  [ 7.050,  0.000,  0.000]
bvect =  [ 0.000,  7.041,  0.000]
cvect =  [ 0.000,  0.000,  7.038]
origin = [-0.005, -0.001,  0.001]
natoms = 32
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.343 |   0.361 |   0.364
      1 |       1 |   2.133 |   2.087 |   0.365
      2 |       1 |   2.130 |   0.356 |   2.106
      3 |       1 |   0.371 |   2.149 |   2.115
      4 |       1 |   3.879 |   0.337 |   0.337
      5 |       1 |   5.617 |   2.130 |   0.353
      6 |       1 |   5.649 |   0.364 |   2.103
      7 |       1 |   3.858 |   2.098 |   2.122
      8 |       1 |   0.384 |   3.867 |   0.325
      9 |       1 |   2.079 |   5.679 |   0.352
     10 |       1 |   2.136 |   3.855 |   2.119
     11 |       1 |   0.368 |   5.614 |   2.116
     12 |       1 |   3.863 |   3.856 |   0.369
     13 |       1 |   5.607 |   5.613 |   0.379
     14 |       1 |   5.657 |   3.848 |   2.129
     15 |       1 |   3.903 |   5.656 |   2.119
     16 |       1 |   0.

Show list of property keys. Note the names of the c_voro and c_stress terms match with how they are represented in the dump file. The user-defined nature of LAMMPS properties and property names means that atomman can't automatically interpret such terms.

In [5]:
print system.atoms_prop()

['atype', 'pos', 'c_voro[1]', 'c_voro[2]', 'c_voro[3]', 'c_stress[1]', 'c_stress[2]', 'c_stress[3]', 'c_stress[4]', 'c_stress[5]', 'c_stress[6]']


Additionally, the timestep information is stored in system.prop dictionary

In [6]:
print system.prop['timestep']

10000


### Default prop_info model

Even though a prop_info data model wasn't specified, one was created and saved to the file test.dump.json. Its contents define the property name conversions that were performed on the data.

Note that the default prop_info created by [atom_dump.dump](atomman.lammps.atom_dump.dump) will not be exactly the same.

In [7]:
with open('test.dump.json') as f:
    print f.read()

{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": null
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "c_voro[1]": {}, 
            "c_voro[2]": {}, 
            "c_voro[3]": {}, 
            "c_stress[1]": {}, 
            "c_stress[2]": {}, 
            "c_stress[3]": {}, 
            "c_stress[4]": {}, 
            "c_stress[5]": {}, 
            "c_stress[6]": {}
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos", 
                    "index": 0
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "index":

Having the default test.dump.json file also ensures that if system is exported back into a dump file using [atom_dump.dump](atomman.lammps.atom_dump.dump) that all the terms are converted back into the original property names. 

In [8]:
lmp.atom_dump.dump(system, 'test.dump', xf='%g', prop_info='test.dump.json')
with open('test.dump') as f:
    print f.read()

ITEM: TIMESTEP
10000
ITEM: NUMBER OF ATOMS
32
ITEM: BOX BOUNDS pp pp pp
-0.005187 7.045190
-0.000725 7.040730
0.001062 7.038940
ITEM: ATOMS id type x y z c_voro[1] c_voro[2] c_voro[3] c_stress[1] c_stress[2] c_stress[3] c_stress[4] c_stress[5] c_stress[6]
1 1 0.34266 0.360855 0.36442 10.9751 15 26.4002 400382 -172369 361946 618490 404704 -320814
2 1 2.1331 2.08739 0.364684 10.8585 14 26.2102 -615610 -230235 -309261 -51268.4 -22109.6 162304
3 1 2.13011 0.356301 2.10647 10.7928 12 26.1077 -348083 -910871 -1.09017e+06 -378108 -14530.8 17524
4 1 0.371264 2.14855 2.11487 10.8736 14 26.2371 -274887 -631508 -200777 91647.5 -110065 42194.8
5 1 3.87914 0.336744 0.33682 10.8095 13 26.1342 -675768 -782205 -574469 280706 387178 -88306.6
6 1 5.61718 2.12982 0.353137 10.9855 15 26.4155 634348 23563.8 352267 410632 -16999 89704.6
7 1 5.64899 0.364151 2.10255 10.8934 13 26.2708 -377029 366661 -568931 -241658 -167956 -98843
8 1 3.85763 2.09815 2.1219 10.8531 15 26.2035 -617312 -504639 -133137 82641.5 -

### Customizing prop_info

The prop_info data model specifies how atomman interprets and converts properties between LAMMPS dump files and System objects. It provides the ability to fully define the metadata associated with a LAMMPS dump file so that terms and values can be appropriately interpreted without knowledge of LAMMPS settings and user-defined properties. And you only have to create one prop_info model for a specific simulation/System!

A demonstration is given below. More information on the meaning of the different terms can be found in the [prop_info Notebook](../data_models/prop_info.ipynb).

Rather than starting from scratch, it is easiest to call atom_data.load to generate the default model, then fill in missing values.  Here, I've adapted the model above by 

1. Giving angstrom units for the box vectors (LAMMPS-dump-atoms_prop-relate -> box_prop -> unit) 
2. Defining more informative names for the voronoi terms (LAMMPS-dump-atoms_prop-relate -> atoms_prop) 
3. Defining a tensor with shape (3,3) for stress (LAMMPS-dump-atoms_prop-relate -> atoms_prop) 
4. Giving appropriate data types for the various property names (LAMMPS-dump-atoms_prop-relate -> atoms_prop -> * -> dtype) 
5. Giving appropriate units for the LAMMPS property names (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute -> * -> relation -> unit) 
6. Relating c_voro terms to the defined property names (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute - > * -> relation -> prop)
7. Relating c_stress to different components of the stress tensor (LAMMPS-dump-atoms_prop-relate -> LAMMPS-attribute - > * -> relation)

In [9]:
from DataModelDict import DataModelDict as DM

In [10]:
prop_info = """{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": "angstrom"
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "voronoi-volume": {
                "dtype": "float"
            }, 
            "voronoi-number-of-surfaces": {
                "dtype": "int"
            }, 
            "voronoi-surface-area": {
                "dtype": "float"
            }, 
            "stress": {
                "dtype": "float",
                "shape": [
                    3,
                    3
                ]
            }
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos",
                    "unit": "angstrom",
                    "index": 0
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "unit": "angstrom",
                    "index": 1
                }
            }, 
            "z": {
                "relation": {
                    "prop": "pos", 
                    "unit": "angstrom",
                    "index": 2
                }
            }, 
            "c_voro[1]": {
                "relation": {
                    "prop": "voronoi-volume",
                    "unit": "angstrom^3"
                }
            }, 
            "c_voro[2]": {
                "relation": {
                    "prop": "voronoi-number-of-surfaces"
                }
            }, 
            "c_voro[3]": {
                "relation": {
                    "prop": "voronoi-surface-area",
                    "unit": "angstrom^2"
                }
            }, 
            "c_stress[1]": {
                "relation": {
                    "prop": "stress",
                    "unit": "bar*angstrom^3",
                    "index": [
                        0,
                        0
                    ]
                }
            }, 
            "c_stress[2]": {
                "relation": {
                   "prop": "stress",
                   "unit": "bar*angstrom^3",
                    "index": [
                        1,
                        1
                    ]
                }
            }, 
            "c_stress[3]": {
                "relation": {
                    "prop": "stress",
                    "unit": "bar*angstrom^3",
                    "index": [
                        2,
                        2
                    ]
                }
            }, 
            "c_stress[4]": {
                "relation": [
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            0,
                            1
                        ]
                    },
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            1,
                            0
                        ]
                    }
                ]
            }, 
            "c_stress[5]": {
                "relation": [
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            0,
                            2
                        ]
                    },
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            2,
                            0
                        ]
                    }
                ]
            }, 
            "c_stress[6]": {
                "relation": [
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            1,
                            2
                        ]
                    },
                    {
                        "prop": "stress",
                        "unit": "bar*angstrom^3",
                        "index": [
                            2,
                            1
                        ]
                    }
                ]
            }
        }
    }
}"""

Now, after reading in test.dump

In [11]:
system = am.lammps.atom_dump.load('test.dump', prop_info=prop_info)

We see that the property names are consistent with how they are defined, 

In [12]:
print system.atoms_prop()

['atype', 'pos', u'voronoi-volume', u'voronoi-number-of-surfaces', u'voronoi-surface-area', u'stress']


and that stress is a symmetric tensor with values changed into atomman's working units

In [13]:
print system.atoms_prop(key='stress', a_id=0)

[[ 0.2498988   0.38603111  0.25259638]
 [ 0.38603111 -0.10758427 -0.20023636]
 [ 0.25259638 -0.20023636  0.22590893]]


__File Cleanup__

In [14]:
os.remove('test.dump')
os.remove('test.dump.json')

- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)